In [ ]:
from langchain.llms import OpenAI
api_key = "sk-pvdcqtmuphqjgepywixzapshexhxcugkhnrtjnuhdotlvgtx"
base_url = "https://api.siliconflow.cn/v1"
llm = OpenAI(
    # model_name="deepseek-ai/DeepSeek-V2.5",
    model = "Qwen/Qwen2.5-7B-Instruct",
    temperature=0,
    stream=False,
    openai_api_base=base_url,
    openai_api_key=api_key
)
result = llm.predict("你是谁？")
print(result)

### 示例选择器
- 根据长度要求智能选择示例
- 根据输入相似度选择示例（最大边际相关性)
- 根据输入相似度选择示例（最大余弦相似度)

### 根据长度要求智能选择示例


In [ ]:
## 根据输入提示词的长度，综合计算最终长度，智能截取或者添加提示词的示例
from langchain.prompts import PromptTemplate,FewShotPromptTemplate,LengthBasedExampleSelector

# 假设已经有这么多的提示词
examples = [
    {"input":"你是谁？","output":"我是一个AI助手"},
    {"input":"你能做什么？","output":"我可以回答你的问题"},
    {"input":"你喜欢吃什么？","output":"我不需要吃东西"},
    {"input":"你不需要电和算力吗","output":"我不需要电和算力"},
]
# 构造提示词模板
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="输入：{input}\n输出：{output}"
)
# 调用长度示例选择器
example_selector = LengthBasedExampleSelector(
    # 传入的提示词示例
    examples=examples,
    # 传入的提示词模板
    example_prompt=example_prompt,
    #设置最大长度
    max_length=5,
)

# 使用小样本提示词模板来实现动态示例的调用 
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="JackLuoAI是一个AI模型，它使用以下示例来回答",
    suffix="输入: {adjective}\n输出：",
    input_variables=["adjective"]
)

print(dynamic_prompt.format(adjective="爱"))

In [ ]:
# 如果输入长度很长，则最终输出会根据长度要求减少
long_string = "This is a very long string that exceeds the maximum length."
print(dynamic_prompt.format(adjective=long_string))


### 根据输入相似度选择示例（最大边际相关性)
- MMR是一种在信息检索中常用的方法，它的目标是在相关性和多样性之间找到一个平衡·
- MMR会首先找出与输入最相似(即余弦相似度最大)的样本
- 然后在迭代添加样本的过程中，对于与已选择样本过于接近(即相似度过高)的样本进行惩罚
- MMR既能确保选出的样本与输入高度相关，又能保证选出的样本之间有足够的多样性
- 关注如何在相关性和多样性之间找到一个平衡

In [41]:
! pip install faiss-cpu
! pip install tiktoken
! pip install chromadb

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ---------------------------------------- 0.0/611.1 kB ? eta -:--:--
     ----------------- ---------------------- 262.1/611.1 kB ? eta -:--:--
     -------------------------------- ----- 524.3/611.1 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 611.1/611.1 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
      --------------------------------------- 0.3/11.3 MB ? eta -:--:--
     - -------------------------------------- 0.5/11.3 MB 1.4 MB/s eta 0:00:08
     -- ------------------------------------- 0.8/11.3 MB 1.4 MB/s eta 0:00:08
     --- ------------------------------------ 1.0/11.3 MB 1.3 MB/s eta 0:00:08
     ---- ----------------------------------- 1.3/11.3 MB 1.3 MB/s eta 0:00:08
     ----- ---------------------------------- 1.6/11

In [ ]:
# 使用MMR 来检索相关示例，以使示例尽量符合输入
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
# 使用FAISS来存储和检索向量 ,需要存储
from langchain.vectorstores import FAISS
# 向量嵌入数据库
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# 假设已经有这么多的提示词
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "fast", "output": "slow"},
    {"input": "hot", "output": "cold"},
    {"input": "smart", "output": "stupid"},
    {"input": "strong", "output": "weak"},
    {"input": "big", "output": "small"},
    {"input": "good", "output": "bad"},
    {"input": "高兴", "output": "悲伤"},
]

# 构造提示词模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="原词：{input}\n反义：{output}"
)

# 初始化向量嵌入
embeddings = OpenAIEmbeddings(
    model="BAAI/bge-m3",
    openai_api_base=base_url,
    openai_api_key=api_key
)

# 初始化FAISS向量存储
vectorstore = FAISS.from_texts(
    [example["input"] for example in examples],
    embedding=embeddings,
    metadatas=examples
)

# 调用 MMR 示例选择器
example_selector = MaxMarginalRelevanceExampleSelector(
    vectorstore=vectorstore,
    k=2
)

# 使用小样本提示词模板来实现动态示例的调用 
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="输入: {adjective}\n输出：",
    input_variables=["adjective"]
)


# 测试
print(dynamic_prompt.format(adjective="高兴"))

给出每个输入词的反义词

原词：高兴
反义：悲伤

原词：windy
反义：calm

输入: 高兴
输出：


### 根据输入相似度选择示例（最大余弦相似度）
- 一种常见的相似度计算方法
- 它通过计算两个向量(在这里，向量可以代表文本、句子或词语)之间的余弦值来衡量它们的相似度
- 余弦值越接近1，表示两个向量越相似
- 主要关注的是如何准确衡量两个向量的相似度

In [45]:
# 使用最大余弦相似度来检索相关示例，以使示例尽量符合输入
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
# 使用Chrom来存储和检索向量 ,需要存储
from langchain.vectorstores import Chroma
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings

# 假设已经有这么多的提示词
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "fast", "output": "slow"},
    {"input": "hot", "output": "cold"},
    {"input": "smart", "output": "stupid"},
    {"input": "strong", "output": "weak"},
    {"input": "big", "output": "small"},
    {"input": "good", "output": "bad"},
    {"input": "高兴", "output": "悲伤"},
]

# 构造提示词模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="原词：{input}\n反义：{output}"
)

# 初始化向量嵌入
embeddings = OpenAIEmbeddings(
    model="BAAI/bge-m3",
    openai_api_base=base_url,
    openai_api_key=api_key
)
vectorstore = Chroma.from_texts(
    [example["input"] for example in examples],
    embedding=embeddings,
    metadatas=examples
)
# 调用 MMR 示例选择器
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2
)

# 使用小样本提示词模板来实现动态示例的调用 
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="输入: {adjective}\n输出：",
    input_variables=["adjective"]
)


# 测试
print(dynamic_prompt.format(adjective="happy"))

给出每个输入词的反义词

原词：happy
反义：sad

原词：happy
反义：sad

输入: happy
输出：
